<a href="https://colab.research.google.com/github/ArtsARKADE/versemagic/blob/main/SPaD/Layer-1/ComparativeJudgement/BCJ_version_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import time

# Define the directory path to save the results
RESULTS_DIR = "/content/drive/MyDrive/PoemSelectionResults/"

def load_poems_from_multiple_paths(paths):
    poems = []
    for path in paths.split(';'):
        if not os.path.exists(path):
            print(f"Path does not exist: {path}")
            continue
        try:
            print(f"Processing path: {path}")
            # Walk through all subdirectories and files
            for root, _, files in os.walk(path):
                print(f"Processing directory: {root}")
                files = [f for f in files if f.endswith('.txt')]
                print(f"Found {len(files)} files in {root}")
                for file in files:
                    full_path = os.path.join(root, file)
                    if not os.path.isfile(full_path):
                        print(f"File not found: {full_path}")
                        continue
                    try:
                        with open(full_path, 'r', encoding='utf-8') as f:
                            content = f.read()
                    except UnicodeDecodeError:
                        try:
                            with open(full_path, 'r', encoding='ISO-8859-1') as f:
                                content = f.read()
                        except Exception as e:
                            print(f"Failed to read {file} with ISO-8859-1: {e}")
                            continue

                    # Extract relevant parts from the cleaned content
                    name_match = re.search(r'Name:\s*(.*)', content)
                    title_match = re.search(r'Title:\s*(.*)', content)
                    poem_text_match = re.search(r'<poem>(.*?)</poem>', content, re.DOTALL)

                    name = name_match.group(1).strip() if name_match else "Unknown Name"
                    title = title_match.group(1).strip() if title_match else "Unknown Title"
                    poem_text = poem_text_match.group(1).strip().split('\n') if poem_text_match else ["No poem text found"]

                    if poem_text != ["No poem text found"]:
                        poems.append({
                            'name': name,
                            'title': title,
                            'text': poem_text
                        })
                        print(f"Loaded poem from {file}: Name='{name}', Title='{title}'")
                    else:
                        print(f"Poem text not found in file {file}")
            print(f"Loaded {len(poems)} poems from {path} successfully.")
        except Exception as e:
            print(f"Failed to process files in {path}: {e}")
    return poems

def create_pairs(poems, num_pairs=10, max_attempts=5):
    pairs = []
    attempt = 0
    if len(poems) < 2:
        print("Not enough poems available to form pairs.")
        return pairs
    while len(pairs) < num_pairs and attempt < max_attempts:
        attempt += 1
        try:
            stanza1, stanza2 = np.random.choice(poems, size=2, replace=False)
            pairs.append((stanza1, stanza2))
        except ValueError:
            continue
    return pairs

# Ensure the results directory exists
if not os.path.exists(RESULTS_DIR):
    os.makedirs(RESULTS_DIR)

# UI Components
path_input = widgets.Textarea(description="Poem Paths:", layout={'width': '500px', 'height': '100px'})
load_button = widgets.Button(description="Load Poems")
results_area = widgets.VBox([])
timer_output = widgets.Output()

# Player Results
player_results = []

# Event Handlers
def on_load_button_clicked(b):
    load_poems_and_display_round()

def load_poems_and_display_round():
    global player_results
    player_results = []
    start_time = time.time()
    display_round("Round One", start_time)

def display_round(round_name, start_time):
    clear_output()
    print(round_name + ":")
    poems = load_poems_from_multiple_paths(path_input.value)
    pairs = create_pairs(poems)
    if not pairs:
        print("Could not find enough pairs after retries.")
    else:
        display_stanza_pairs(pairs, start_time)

def display_stanza_pairs(pairs, start_time):
    global results_area
    results_area = widgets.VBox([])  # Reinitialize results_area as a VBox
    if not pairs:
        display(HTML("<b>No pairs available, check the number of valid poems.</b>"))
    else:
        for index, (stanza1, stanza2) in enumerate(pairs):
            text_area1 = widgets.Textarea(value='\n'.join(stanza1['text'][:2]), layout={'height': '100px', 'width': '250px'}, disabled=True)
            text_area2 = widgets.Textarea(value='\n'.join(stanza2['text'][:2]), layout={'height': '100px', 'width': '250px'}, disabled=True)
            button1 = widgets.Button(description="Select Poem A")
            button2 = widgets.Button(description="Select Poem B")
            button1.on_click(lambda b, idx=index, selected_stanza=stanza1, other_stanza=stanza2, start_time=start_time: select_stanza(idx, selected_stanza, other_stanza, start_time))
            button2.on_click(lambda b, idx=index, selected_stanza=stanza2, other_stanza=stanza1, start_time=start_time: select_stanza(idx, selected_stanza, other_stanza, start_time))
            row = widgets.HBox([widgets.Label('Poem A'), text_area1, button1, widgets.Label('Poem B'), text_area2, button2])
            results_area.children += (row,)  # Append each row to the results_area
        display(results_area)

def select_stanza(idx, selected_stanza, other_stanza, start_time):
    end_time = time.time()
    total_time = end_time - start_time
    player_results.append((selected_stanza['name'], selected_stanza['title'], '\n'.join(selected_stanza['text']), other_stanza['name'], other_stanza['title'], total_time))
    if len(player_results) == 5:
        print("End of Rounds. Writing results...")
        write_results(player_results)
    else:
        display_round(f"Round {len(player_results) + 1}", start_time)

def write_results(results):
    timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
    filename = os.path.join(RESULTS_DIR, f"poem_selection_results_{timestamp}.txt")
    with open(filename, "w") as file:
        file.write("Poem Selection Results:\n")
        for i, (selected_name, selected_title, selected_text, other_name, other_title, total_time) in enumerate(results):
            file.write(f"Round {i+1}: Selected '{selected_name}' - '{selected_title}' over '{other_name}' - '{other_title}'. Total time: {total_time:.2f} seconds.\n")
            file.write(f"Selected Poem Text:\n{selected_text}\n")
    print("Results written to", filename)

load_button.on_click(on_load_button_clicked)

# Layout and Display
ui = widgets.VBox([
    path_input,
    load_button,
    results_area,
    timer_output
])

display(ui)